In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/Books.csv


In [2]:

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import estimator_html_repr

import joblib
from IPython.core.display import HTML

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers, utils, optimizers, callbacks

# Read dataset

In [3]:
books = pd.read_csv("/kaggle/input/book-recommendation-dataset/Books.csv", delimiter=",",usecols = [0,1,2,3,4])
users = pd.read_csv("/kaggle/input/book-recommendation-dataset/Users.csv")
ratings = pd.read_csv("/kaggle/input/book-recommendation-dataset/Ratings.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# EDA

In [4]:
print(books.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271359 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB
None


In [108]:
print(users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB
None


In [109]:
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB
None


In [110]:
#no need
df_books=books.iloc[:1000]
print(df_books.head(5))

         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  
0    Mark P. O. Morford                2002     Oxford University Press  
1  Richard Bruce Wright                2001       HarperFlamingo Canada  
2          Carlo D'Este                1991             HarperPerennial  
3      Gina Bari Kolata                1999        Farrar Straus Giroux  
4       E. J. W. Barber                1999  W. W. Norton &amp; Company  


In [111]:
#no need
print(df_books.shape)

(1000, 5)


In [112]:
#no need
print(df_books.isnull().sum())

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
dtype: int64


In [5]:
ratings_with_name = ratings.merge(books,on='ISBN')
(ratings_with_name.head())

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books


In [114]:
ratings_with_name.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031136 entries, 0 to 1031135
Data columns (total 7 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   User-ID              1031136 non-null  int64 
 1   ISBN                 1031136 non-null  object
 2   Book-Rating          1031136 non-null  int64 
 3   Book-Title           1031136 non-null  object
 4   Book-Author          1031135 non-null  object
 5   Year-Of-Publication  1031136 non-null  object
 6   Publisher            1031134 non-null  object
dtypes: int64(2), object(5)
memory usage: 62.9+ MB


In [6]:
complete_df = ratings_with_name.merge(users.drop("Age", axis=1), on="User-ID")
complete_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,"tyler, texas, usa"
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,"cincinnati, ohio, usa"
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,"cincinnati, ohio, usa"
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,"cincinnati, ohio, usa"
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,"cincinnati, ohio, usa"


In [116]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031136 entries, 0 to 1031135
Data columns (total 8 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   User-ID              1031136 non-null  int64 
 1   ISBN                 1031136 non-null  object
 2   Book-Rating          1031136 non-null  int64 
 3   Book-Title           1031136 non-null  object
 4   Book-Author          1031135 non-null  object
 5   Year-Of-Publication  1031136 non-null  object
 6   Publisher            1031134 non-null  object
 7   Location             1031136 non-null  object
dtypes: int64(2), object(6)
memory usage: 70.8+ MB


In [7]:
#only including country in location
complete_df['Location'] = complete_df['Location'].str.split(',').str[-1].str.strip()

# Preprocessing

In [8]:
#it was found later that year of publication had text
complete_df['Year-Of-Publication'] = pd.to_numeric(complete_df['Year-Of-Publication'],errors='coerce')
complete_df = complete_df.dropna()
complete_df['Year-Of-Publication'] = complete_df['Year-Of-Publication'].astype(int)
complete_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,usa
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,usa
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,usa


In [129]:
print('Number of data before cleaning : {}'.format(len(complete_df)))


Number of data before cleaning : 428085


In [9]:
f = ['count','mean']

df_books_summary = complete_df.groupby('ISBN')['Book-Rating'].agg(f)
df_books_summary.index = df_books_summary.index.map(str)

drop_book_list = df_books_summary[df_books_summary['count'] < 10].index

df_cust_summary = complete_df.groupby('User-ID')['Book-Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)

drop_cust_list = df_cust_summary[df_cust_summary['count'] < 10].index

In [10]:
print('Before Filtering: {}'.format(complete_df.shape))
complete_df = complete_df[~complete_df['ISBN'].isin(drop_book_list)]
complete_df = complete_df[~complete_df['User-ID'].isin(drop_cust_list)]
print('After Filtering: {}'.format(complete_df.shape))

Before Filtering: (1031129, 8)
After Filtering: (428085, 8)


In [121]:
complete_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,usa
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,usa
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,usa
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,usa
5,2313,0385482388,5,The Mistress of Spices,Chitra Banerjee Divakaruni,1998,Anchor Books/Doubleday,usa


# Content based Recommation

This recommendation is made based on an assumption that a certain reader will be attracted to read another book based factors below:
- authors
Some readers might be attracted to some certain authors because some authors tends to write about a specific genre that certain demographic of reader like. So, including auhtor as one of the factor might be plausible.

- year-of-publication
Year of publication might reflect some kind of genre preferred by readers (e.g. reader who would like to read classical philosophy books might tend to read older books compared to new ones).


- publisher
Certain publisher has a certain kind of credibility which might make some book readers prefer one publisher compared to other.

Based on that consideration, we will consider Author, Year of publication, as well as publisher as basis for recommendation.

In [11]:
len(complete_df['ISBN'].unique())

17478

In [12]:
#These are the features we have selected
features = ['Book-Author', 'Year-Of-Publication', 'Publisher']

df_books_preprocess = complete_df[complete_df['ISBN'].isin(complete_df['ISBN'].unique())].copy()


for feature in features:
    if df_books_preprocess[feature].dtype == 'O':
        df_books_preprocess[feature]=df_books_preprocess[feature].str.replace('\W', '', regex=True)
        df_books_preprocess[feature] =  df_books_preprocess[feature].apply(lambda x: str.lower(x))

In [13]:
#creating a string of features to calculate similarity
def create_soup(x):
    return x['Book-Author'] + ' ' + str(x['Year-Of-Publication'])+ ' ' + x['Publisher']

df_books_preprocess['soup'] = df_books_preprocess[features].apply(create_soup, axis = 1)
df_books_preprocess.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,soup
1,2313,034545104X,5,Flesh Tones: A Novel,mjrose,2002,ballantinebooks,usa,mjrose 2002 ballantinebooks
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),orsonscottcard,1986,torbooks,usa,orsonscottcard 1986 torbooks
3,2313,0679745580,8,In Cold Blood (Vintage International),trumancapote,1994,vintage,usa,trumancapote 1994 vintage
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,rebeccawells,1996,harpercollins,usa,rebeccawells 1996 harpercollins
5,2313,0385482388,5,The Mistress of Spices,chitrabanerjeedivakaruni,1998,anchorbooksdoubleday,usa,chitrabanerjeedivakaruni 1998 anchorbooksdoubl...


In [14]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df_books_preprocess['soup'])

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
df2 = df_books_preprocess.reset_index()
indices = pd.Series(df2.index, index=df2['ISBN'])